In [1]:
from os import listdir, walk
from os.path import isfile, join
import numpy as np
from PIL import Image
import cv2
from IPython.display import display
import math, random
import time, datetime, sys, os, shutil
import operator

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
import pandas as pd

print(tf.config.experimental.list_physical_devices())

BATCH_SIZE = 32

%run ./variables.ipynb
%run ./utils.ipynb
%run ../utils/data_utils.ipynb

id_map = get_selected_taxons(SELECTED_TAXONS)
id_map_inv = get_selected_taxons(SELECTED_TAXONS, inv=True)
n_classes = len(list(id_map.keys()))
check_dirs(SAVED_MODELS_ROOT)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Handle data

## Preparing panda arrays

In [2]:
X, y, _ = get_dataset(DATASET_PATH, ids=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
delete_all_files_in_folder(SAVED_MODELS_ROOT)
array_to_csv(X_train, os.path.join(SAVED_MODELS_ROOT, "train_list.csv"))
array_to_csv(X_test, os.path.join(SAVED_MODELS_ROOT, "test_list.csv"))

print("Test length:", len(X_train), "- n classes:", len(np.unique(y_train)))
print("Test length:", len(X_test), "- n classes:", len(np.unique(y_test)))

# Balance dataset
X_train, y_train, max_samples = balance_dataset(X_train, y_train, max_samples=None)

Retrieving dataset from: /mnt/nvme-storage/pfauregi/training/thumbails/atlas/dataset/


'166/166'

Deleting all files in /mnt/nvme-storage/pfauregi/training/thumbails/atlas/saved_models/
Creating: /mnt/nvme-storage/pfauregi/training/thumbails/atlas/saved_models/train_list.csv
Creating: /mnt/nvme-storage/pfauregi/training/thumbails/atlas/saved_models/test_list.csv
Test length: 8905 - n classes: 166
Test length: 990 - n classes: 166
Balanced to 166 samples per class!


In [3]:
data_train = {'png_path':  X_train, 'taxon_id': y_train}
data_test = {'png_path':  X_test, 'taxon_id': y_test}

df_train = pd.DataFrame(data_train, columns = ['png_path', 'taxon_id'])
df_test = pd.DataFrame(data_test, columns = ['png_path', 'taxon_id'])
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

# Prtining some infos
print(len(df_train), len(df_test))
df_train.head()
df_test.head()

27556 990


,png_path,taxon_id
0,/mnt/nvme-storage/pfauregi/training/thumbails/...,EOCO
1,/mnt/nvme-storage/pfauregi/training/thumbails/...,FNEV
2,/mnt/nvme-storage/pfauregi/training/thumbails/...,RSIN
3,/mnt/nvme-storage/pfauregi/training/thumbails/...,CDUB
4,/mnt/nvme-storage/pfauregi/training/thumbails/...,NROS


## Setting up the flows

In [4]:
def preproc(img):
    # Zoom img
    zoomed_img=cv2_clipped_zoom(img, np.random.uniform(80,110)/100)
    # Shift img
    ox, oy = np.random.randint(-20,20,2)/100
    return tf.keras.preprocessing.image.random_shift(zoomed_img, ox, oy, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')

train_datagen = ImageDataGenerator(rescale=1./255.,
                         rotation_range=180, 
                         brightness_range=[0.8,1.2], 
                         horizontal_flip=True, 
                         vertical_flip=True,
                         fill_mode='nearest',
                         preprocessing_function=preproc,
                         #width_shift_range=10,
                         #height_shift_range=10,
                         #zoom_range=[-0.2,0.2],
                         data_format="channels_last")

test_datagen = ImageDataGenerator(rescale=1./255.,
                         data_format="channels_last")

In [5]:
#classes_array = np.unique(y_train)
train_generator = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        #classes=classes_array,
        class_mode='categorical')
val_generator = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        classes=train_generator.class_indices,
        class_mode='categorical')

train_spe = train_generator.samples // BATCH_SIZE
val_spe = val_generator.samples // BATCH_SIZE

Found 27556 validated image filenames belonging to 166 classes.
Found 990 validated image filenames belonging to 166 classes.


In [6]:
class_indices = train_generator.class_indices
f = open(os.path.join(SAVED_MODELS_ROOT, 'model_id_map.csv'), 'w')
with f:
    writer = csv.writer(f)
    writer.writerow(["taxon", "id"])
    for taxon in class_indices:
        writer.writerow([taxon, class_indices[taxon]])

## Testing

In [ ]:
i = 0
stop = False
for batch in val_generator:
    images = batch[0]
    labels = batch[1]
    for i in range(images.shape[0]):
        print(np.argmax(labels[i]))
        image = (images[i,:,:,:]*255).astype('uint8')
        #print(image)
        display(Image.fromarray(image))
        i+=1
        if i>=1:
            stop = True
            break
    if stop: pass;
    #display()

# Model desgin

In [7]:
# fetching base model
input_tensor = Input(shape=(256, 256, 3))
#base_model = InceptionV3(weights='imagenet', input_tensor=input_tensor, include_top=False)
base_model = Xception(include_top=False, weights='imagenet', input_tensor=input_tensor, pooling=None)

In [8]:
# setting model for specifiv case
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
out = Dense(len(train_generator.class_indices.keys()), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

# Training

## Setting callbacks

In [9]:
# Setting tensorboard
check_dirs(LOG_DIR)
delete_all_files_in_folder(LOG_DIR)
log_dir = LOG_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

Deleting all files in /mnt/nvme-storage/pfauregi/training/thumbails/atlas/tensorboard/


In [10]:
log_file=os.path.join(SAVED_MODELS_ROOT, "model.log")
#os.remove(log_file)
csv_logger = CSVLogger(log_file, append=True)
checkpointer = ModelCheckpoint(filepath=os.path.join(SAVED_MODELS_ROOT, "weights.hdf5"), verbose=True, save_best_only=True)

In [16]:
model_path=os.path.join(SAVED_MODELS_ROOT, "model.json")
model_json = model.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_json)

## Setting optimizer

In [11]:
#optimizer = SGD(lr=0.1, decay=0.0001, momentum=0, nesterov=False)
optimizer = "adam"

## New layers only

In [12]:
#n_epochs = 5
n_epochs = 3
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger, checkpointer],
          initial_epoch=last_epoch)

3 epochs composed of 861 batches (steps) of 32 images.
Epoch 1/3
861/861 [==============================] - ETA: 0s - loss: 1.9198 - accuracy: 0.5006
Epoch 00001: val_loss improved from inf to 1.54817, saving model to weights.hdf5
861/861 [==============================] - 594s 690ms/step - loss: 1.9198 - accuracy: 0.5006 - val_loss: 1.5482 - val_accuracy: 0.5323
Epoch 2/3
861/861 [==============================] - ETA: 0s - loss: 1.0183 - accuracy: 0.6859
Epoch 00002: val_loss improved from 1.54817 to 1.26597, saving model to weights.hdf5
861/861 [==============================] - 581s 675ms/step - loss: 1.0183 - accuracy: 0.6859 - val_loss: 1.2660 - val_accuracy: 0.6333
Epoch 3/3
861/861 [==============================] - ETA: 0s - loss: 0.8118 - accuracy: 0.7482
Epoch 00003: val_loss improved from 1.26597 to 1.19535, saving model to weights.hdf5
861/861 [==============================] - 580s 673ms/step - loss: 0.8118 - accuracy: 0.7482 - val_loss: 1.1953 - val_accuracy: 0.6354


In [13]:
np.max(history.history["val_accuracy"])

0.6354166865348816

## Training full model

In [15]:
#n_epochs = 50
n_epochs = 40
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger, checkpointer],
          initial_epoch=last_epoch)
print(np.max(history.history["val_accuracy"]))

40 epochs composed of 861 batches (steps) of 32 images.
Epoch 8/47
  2/861 [..............................] - ETA: 3:14 - loss: 0.3387 - accuracy: 0.8889WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.196253). Check your callbacks.


861/861 [==============================] - ETA: 0s - loss: 0.2690 - accuracy: 0.9133
Epoch 00008: val_loss improved from 1.19535 to 0.85782, saving model to weights.hdf5
861/861 [==============================] - 589s 685ms/step - loss: 0.2690 - accuracy: 0.9133 - val_loss: 0.8578 - val_accuracy: 0.8125
Epoch 9/47
861/861 [==============================] - ETA: 0s - loss: 0.2231 - accuracy: 0.9269
Epoch 00009: val_loss improved from 0.85782 to 0.72377, saving model to weights.hdf5
861/861 [==============================] - 586s 681ms/step - loss: 0.2231 - accuracy: 0.9269 - val_loss: 0.7238 - val_accuracy: 0.8354
Epoch 10/47
861/861 [==============================] - ETA: 0s - loss: 0.1967 - accuracy: 0.9373
Epoch 00010: val_loss did not improve from 0.72377
861/861 [==============================] - 586s 681ms/step - loss: 0.1967 - accuracy: 0.9373 - val_loss: 0.9314 - val_accuracy: 0.7854
Epoch 11/47
861/861 [==============================] - ETA: 0s - loss: 0.1731 - accuracy: 0.9440

KeyboardInterrupt: 

## Saving model

In [ ]:
save_model(model, SAVED_MODELS_ROOT)